In [1]:
# Initialize the relavent data structure
flight = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6']
flight_time = {'AUS,DAL': 50, 'DAL,AUS': 50, 'AUS,HOU': 45, 'HOU,AUS': 45, 'DAL,HOU': 65, 'HOU,DAL': 65}
airport_gate_groundtime = {'AUS': 25, 'DAL': 30, 'HOU': 35}

# Initialize the starting status
flight_status_table = {'T1': 360, 'T2': 360, 'T3': 360, 'T4': 360, 'T5': 360, 'T6': 360}
flight_gate_time = {'T1':['AUS1',360], 'T2':['DAL1',360], 'T3':['DAL2',360], 'T4':['HOU1',360], 'T5':['HOU2',360], 'T6':['HOU3',360]}
Gate_avail_time_and_sched_ct = {'AUS1': [360,0], 'DAL1': [360,0], 'DAL2': [360,0], 'HOU1': [360,0], 'HOU2': [360,0], 'HOU3': [360,0]}

# Function for Military Time to Minutes since Midnight
def Military2Minutes(Military_Time):
    Military_Time_str = str(Military_Time).zfill(4)
    return 60 * int(Military_Time_str[0:2]) + int(Military_Time_str[2:4])

# Function for Minutes since Midnight to Military Time
def Minutes2Military(Minutes_since_Midnight):
    return str(int(Minutes_since_Midnight / 60)).zfill(2) + str(Minutes_since_Midnight % 60).zfill(2)

# Function for export flight schedule
def ExportSchedule(flight_number, Dep_port, Arr_port, start, end):
    itinerary = flight_number + ' ' + Dep_port + ' ' + Arr_port + ' ' + Minutes2Military(start) + ' ' + Minutes2Military(end)
    print(itinerary)

# Function for decide which airport to go and update airport status at the same time
def Airport2Go(current_time, flight_time, flight_number, airport_gate_groundtime, flight_gate_time, flight_status_table, Gate_avail_time_and_sched_ct):
    Departure_Airport = [value for key, value in flight_gate_time.items() if key == flight_number][0][0][0:3]
    # Departure_Airport = [value for key, value in flight_gate_time.items() if key == flight_number][0][0]
    fly_ground_time = {}
    Temp_airport_gate_time = {}
    Success_schedule = 0
    Candidate_count = 0
    # Identify the potential airport gate that can be selected to fly
    for key,value in flight_time.items():
        if key[0:3] == Departure_Airport and Success_schedule == 0:
        # if key[0:3] == Departure_Airport[0:3] and Success_schedule == 0:
            fly_ground_time[key] = flight_time[key] + airport_gate_groundtime[key[4:7]]
            for key2,value2 in Gate_avail_time_and_sched_ct.items():
                if key2[0:3] == key[4:7] and current_time + fly_ground_time[key] >= Gate_avail_time_and_sched_ct[key2][0] and current_time + flight_time[key] <= 1321:
                    Candidate_count = Candidate_count + 1
                    Temp_airport_gate_time[key2] = current_time + fly_ground_time[key] + 1
    # print('Gate status:', Gate_avail_time_and_sched_ct)
    if Candidate_count >= 1:
        if ('HOU1' or 'HOU2' or 'HOU3') in Temp_airport_gate_time.keys():
            Temp_HOU_gate = []
            for key in Temp_airport_gate_time.keys():
                if key.find("HOU") != -1:
                    Temp_HOU_gate.append(key)
            Min_count = min([Gate_avail_time_and_sched_ct[key][1] for key in Temp_HOU_gate])
            Temp = [key for key, value in Gate_avail_time_and_sched_ct.items() if Gate_avail_time_and_sched_ct[key][1] == Min_count and key in Temp_HOU_gate]
        else:
            Min_count = min([Gate_avail_time_and_sched_ct[key][1] for key in Temp_airport_gate_time.keys()])
            Temp = [key for key,value in Gate_avail_time_and_sched_ct.items() if Gate_avail_time_and_sched_ct[key][1] == Min_count]
        Min_time = min([value for key,value in Temp_airport_gate_time.items() if key in Temp])
        # print('Potential airport:',Temp_airport_gate_time)
        key2 = [key for key,value in Temp_airport_gate_time.items() if value == Min_time and key in Temp][0]
        # print('Arrival airport:', key2)
        Gate_avail_time_and_sched_ct[key2][0] = Temp_airport_gate_time[key2]
        Gate_avail_time_and_sched_ct[key2][1] += 1
        flight_status_table[flight_number] = Temp_airport_gate_time[key2]
        flight_gate_time[flight_number] = [key2, Temp_airport_gate_time[key2]]
        # ExportSchedule(flight_number, Departure_Airport,key2[0:3], current_time, current_time+fly_ground_time[key])
        Success_schedule = 1
        ExportSchedule(flight_number, Departure_Airport[0:3], key2[0:3], current_time, Temp_airport_gate_time[key2]-airport_gate_groundtime[key2[0:3]]-1)
        # ExportSchedule(flight_number, Departure_Airport, key2, current_time, Temp_airport_gate_time[key2] - 1)
    if Success_schedule == 0:
        print(Gate_avail_time_and_sched_ct)
        flight_status_table[flight_number] += 1
        Gate_avail_time_and_sched_ct[Departure_Airport[0:3]][0] += 1

# Start Main Process
current_time = 360
for current_time in range(360,1256):
    # for flight_number,flight_status in flight_status_table.items():
    for flight_number in flight:
        if current_time >= flight_status_table[flight_number]:
            Airport2Go(current_time, flight_time, flight_number, airport_gate_groundtime, flight_gate_time, flight_status_table, Gate_avail_time_and_sched_ct)

print(Gate_avail_time_and_sched_ct)

T1 AUS HOU 0600 0645
T2 DAL HOU 0600 0705
T3 DAL HOU 0600 0705
T4 HOU AUS 0600 0645
T5 HOU DAL 0600 0705
T6 HOU DAL 0600 0705
T4 AUS HOU 0711 0756
T1 HOU AUS 0721 0806
T5 DAL HOU 0736 0841
T6 DAL HOU 0736 0841
T2 HOU DAL 0741 0846
T3 HOU DAL 0741 0846
T1 AUS HOU 0832 0917
T4 HOU AUS 0832 0917
T2 DAL HOU 0917 1022
T3 DAL HOU 0917 1022
T5 HOU DAL 0917 1022
T6 HOU DAL 0917 1022
T4 AUS HOU 0943 1028
T1 HOU AUS 0953 1038
T5 DAL HOU 1053 1158
T6 DAL HOU 1053 1158
T2 HOU DAL 1058 1203
T3 HOU DAL 1058 1203
T1 AUS HOU 1104 1149
T4 HOU AUS 1104 1149
T4 AUS HOU 1215 1300
T1 HOU DAL 1225 1330
T2 DAL HOU 1234 1339
T3 DAL HOU 1234 1339
T5 HOU DAL 1234 1339
T6 HOU AUS 1234 1319
T4 HOU DAL 1336 1441
T6 AUS HOU 1345 1430
T1 DAL HOU 1401 1506
T5 DAL HOU 1410 1515
T2 HOU DAL 1415 1520
T3 HOU AUS 1415 1500
T6 HOU DAL 1506 1611
T4 DAL HOU 1512 1617
T3 AUS HOU 1526 1611
T1 HOU DAL 1542 1647
T2 DAL HOU 1551 1656
T5 HOU AUS 1551 1636
T6 DAL HOU 1642 1747
T3 HOU DAL 1647 1752
T4 HOU DAL 1653 1758
T5 AUS HOU 17